In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [156]:
import pandas as pd
import numpy as np
filename = '/content/drive/MyDrive/DSDojo/PandasForDataManipulation/sales_predictions.csv'
dfsales = pd.read_csv(filename)

# **Data Cleansing**

In [78]:
dfsales.head()


In [33]:
#remove Duplicates if any:
dfsales.duplicated().any()
# No duplicates in data

False

In [ ]:
dfsales.dtypes

In [ ]:
#check for odd data types:
dfsales['Item_Outlet_Sales'].sort_values(ascending = False)
#after looking through each item, no mixed data types were found in any of the columns

In [75]:
#Check for spelling errors or abbreviations etc in each categorical feature:
dfsales['Outlet_Type'].value_counts()
#Issues only found with one feature

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [158]:
#Issues found with Item Fat Content. Fix below:
dfsales.replace({'Item_Fat_Content':{'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'}}, inplace=True)
# Check with: dfsales['Item_Fat_Content'].value_counts()

In [159]:
# Try to see if missing values can be replaced with logical values if any pattern is identified:
dfsales.info()
dfsales.groupby(['Outlet_Identifier','Outlet_Size','Outlet_Location_Type'])[['Outlet_Size']].count()
#No pattern Identified for the Outlet_Size nor the Item_Weight features

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


,,,Outlet_Size
Outlet_Identifier,Outlet_Size,Outlet_Location_Type,
OUT013,High,Tier 3,932
OUT018,Medium,Tier 3,928
OUT019,Small,Tier 1,528
OUT027,Medium,Tier 3,935
OUT035,Small,Tier 2,930
OUT046,Small,Tier 1,930
OUT049,Medium,Tier 1,930


In [160]:
#Create Flag columns for the missing values:
dfsalesf = dfsales
dfsalesf['No_Item_Weight'] = np.where(dfsalesf['Item_Weight'].isnull(), True, False)
dfsalesf['No_Outlet_Size'] = np.where(dfsalesf['Outlet_Size'].isnull(), True, False)
dfsalesf

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,No_Item_Weight,No_Outlet_Size
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,False,False
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,False,False
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,False,False
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,False,True
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834,False,False
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850,False,True
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136,False,False
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976,False,False


In [161]:
dfsales['Outlet_Size'].value_counts(dropna = False)

Medium    2793
NaN       2410
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [92]:
#Handeling missing data
#There are too many missing data too use removal as a resolution. 
#Data for the outlet size is MAR. There are certain stores where ALL their data fro outloet size is missing
#Possible solution for outlet size missing values: 1. Create a 'no size indicated' category/feature value 2. impute the mode (most common value). Though this would
# be assuming all the stores are the same size... which is a stretch 3. Look into KNN if thats an option here or hot-deck

#Data for Item weight is MAR, all weights for items from stores established in 1985 are missing, none are tier 2 adn none of them are from a large outlet
#possible solutions. 1. Since we have data for multiple years succeeding 1985, we could consider dropping the values as we should still be able to predict sales values 
#in the future using what we have, however, roughly ~1500 (17%) of the data would then be dropped. 2. We could use MICE ti impute the missing data

In [162]:
#Some simple sample statistics:
#Items with the highest MRP in each type category:
dfsalesf.groupby(['Item_Type'])['Item_Identifier','Item_MRP'].max()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


,Item_Identifier,Item_MRP
Item_Type,,
Baking Goods,FDZ60,265.5568
Breads,FDZ35,263.6594
Breakfast,FDR37,234.9300
Canned,FDZ49,266.8884
Dairy,FDZ50,266.6884
Frozen Foods,FDZ52,264.8910
Fruits and Vegetables,FDZ56,264.2252
Hard Drinks,DRQ35,261.4278
Health and Hygiene,NCZ53,266.6884


In [103]:
#

Item_Type
Baking Goods             265.5568
Breads                   263.6594
Breakfast                234.9300
Canned                   266.8884
Dairy                    266.6884
Frozen Foods             264.8910
Fruits and Vegetables    264.2252
Hard Drinks              261.4278
Health and Hygiene       266.6884
Household                264.7910
Meat                     261.5936
Others                   254.9040
Seafood                  240.2880
Snack Foods              264.9568
Soft Drinks              263.5910
Starchy Foods            263.0252
Name: Item_MRP, dtype: float64

In [155]:
#Average Sales per Outlet:
dfsalesf.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].mean().sort_values(ascending = False)

Outlet_Identifier
OUT027    3694.038558
OUT035    2438.841866
OUT049    2348.354635
OUT017    2340.675263
OUT013    2298.995256
OUT046    2277.844267
OUT045    2192.384798
OUT018    1995.498739
OUT019     340.329723
OUT010     339.351662
Name: Item_Outlet_Sales, dtype: float64

In [164]:
#Show average Sales per item type over the years
dfsalesf.groupby(['Item_Type','Outlet_Establishment_Year'])[['Item_Outlet_Sales']].mean()

Item_Outlet_Sales
Item_Type     Outlet_Establishment_Year                   
Baking Goods  1985                             2127.730350
              1987                             2050.901134
              1997                             2281.267334
              1998                              254.605090
              1999                             2110.051342
...                                                    ...
Starchy Foods 1999                             2471.762918
              2002                             2396.671937
              2004                             3049.784505
              2007                             2438.529489
              2009                             1947.699988

[144 rows x 1 columns]

In [114]:
# Item category giving the most sales in each year
dfsalesf.groupby(['Outlet_Establishment_Year'])['Item_Outlet_Sales','Item_Type'].max()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Item_Outlet_Sales,Item_Type
Outlet_Establishment_Year,,
1985,13086.9648,Starchy Foods
1987,10256.6490,Starchy Foods
1997,9779.9362,Starchy Foods
1998,1775.6886,Starchy Foods
1999,7646.0472,Starchy Foods
2002,8994.9580,Starchy Foods
2004,8479.6288,Starchy Foods
2007,9664.7528,Starchy Foods
2009,6768.5228,Starchy Foods


In [115]:
# Item category giving the least sales in each year
dfsalesf.groupby(['Outlet_Establishment_Year'])['Item_Outlet_Sales','Item_Type'].min()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Item_Outlet_Sales,Item_Type
Outlet_Establishment_Year,,
1985,33.9558,Baking Goods
1987,73.2380,Baking Goods
1997,101.8674,Baking Goods
1998,33.2900,Baking Goods
1999,111.8544,Baking Goods
2002,99.8700,Baking Goods
2004,113.8518,Baking Goods
2007,143.8128,Baking Goods
2009,69.2432,Baking Goods


In [117]:
#Number of small, med and large outlets over the years:
dfsalesf.groupby(['Outlet_Establishment_Year','Outlet_Size'])['Outlet_Size'].count()

Outlet_Establishment_Year  Outlet_Size
1985                       Medium         935
                           Small          528
1987                       High           932
1997                       Small          930
1999                       Medium         930
2004                       Small          930
2009                       Medium         928
Name: Outlet_Size, dtype: int64

In [129]:
#Number of outlet types over the years:
dfsalesf.groupby(['Outlet_Establishment_Year','Outlet_Type'])[['Outlet_Type']].count()

Outlet_Type
Outlet_Establishment_Year Outlet_Type                   
1985                      Grocery Store              528
                          Supermarket Type3          935
1987                      Supermarket Type1          932
1997                      Supermarket Type1          930
1998                      Grocery Store              555
1999                      Supermarket Type1          930
2002                      Supermarket Type1          929
2004                      Supermarket Type1          930
2007                      Supermarket Type1          926
2009                      Supermarket Type2          928

In [165]:
#Total Cumulative sales over the years: #How do I get it to show the years value????
dfsalesff = dfsalesf
#dfsalesff['Outlet_Establishment_Year']= str(dfsalesff['Outlet_Establishment_Year'])
dfsalesff.groupby(['Outlet_Establishment_Year'])[['Item_Outlet_Sales']].cumsum()#.sort_values(ascending = False)

,Item_Outlet_Sales
0,3.735138e+03
1,4.434228e+02
2,5.832408e+03
3,7.323800e+02
4,9.947052e+02
...,...
8518,2.142664e+06
8519,2.036725e+06
8520,2.268123e+06
8521,1.851823e+06
